In [6]:
import mediapipe as mp
import cv2
import numpy as np

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

# ─── Finger State Helper ─────────────────────────────────
def get_finger_states(landmarks):
    """Returns [thumb, index, middle, ring, pinky] — 1=extended, 0=curled"""
    lm = np.array(landmarks)
    fingers = []

    # Thumb (compare x axis since it moves sideways)
    fingers.append(1 if lm[4][0] > lm[3][0] else 0)

    # Other 4 fingers (tip y < pip y means extended, since y increases downward)
    for tip, pip in [(8,6), (12,10), (16,14), (20,18)]:
        fingers.append(1 if lm[tip][1] < lm[pip][1] else 0)

    return fingers

def get_finger_angles(landmarks):
    finger_joints = [
        [1,2,3], [2,3,4],
        [5,6,7], [6,7,8],
        [9,10,11], [10,11,12],
        [13,14,15], [14,15,16],
        [17,18,19], [18,19,20]
    ]
    lm = np.array(landmarks)
    angles = []
    for a_idx, b_idx, c_idx in finger_joints:
        a, b, c = lm[a_idx], lm[b_idx], lm[c_idx]
        ba, bc = a - b, c - b
        cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        angles.append(np.degrees(np.arccos(np.clip(cosine, -1, 1))))
    return angles

# ─── ASL Sign Classifier ─────────────────────────────────
def classify_sign(landmarks):
    lm = np.array(landmarks)
    fingers = get_finger_states(landmarks)
    angles = get_finger_angles(landmarks)
    thumb, index, middle, ring, pinky = fingers

    # ── ASL 'A' ── fist with thumb to the side
    if thumb == 1 and index == 0 and middle == 0 and ring == 0 and pinky == 0:
        return "A", (0, 200, 255)

    # ── ASL 'B' ── all fingers up, thumb tucked
    if thumb == 0 and index == 1 and middle == 1 and ring == 1 and pinky == 1:
        return "B", (0, 255, 0)

    # ── ASL 'C' ── curved hand (all fingers partially bent)
    avg_angle = np.mean(angles[2:])  # ignore thumb angles
    if 60 < avg_angle < 130 and thumb == 1:
        return "C", (255, 165, 0)

    # ── ASL 'L' ── index up + thumb out, rest curled
    if thumb == 1 and index == 1 and middle == 0 and ring == 0 and pinky == 0:
        return "L", (255, 0, 255)

    # ── ASL 'V' ── index + middle up (peace sign)
    if thumb == 0 and index == 1 and middle == 1 and ring == 0 and pinky == 0:
        return "V", (0, 255, 255)

    # ── ASL 'W' ── index + middle + ring up
    if thumb == 0 and index == 1 and middle == 1 and ring == 1 and pinky == 0:
        return "W", (100, 255, 100)

    # ── ASL 'D' ── index up, others touching thumb
    if index == 1 and middle == 0 and ring == 0 and pinky == 0 and thumb == 1:
        # D and L are similar — differentiate by thumb angle
        thumb_angle = angles[0]
        if thumb_angle > 40:
            return "D", (200, 100, 255)

    # ── Open Hand / '5' ── all fingers extended
    if thumb == 1 and index == 1 and middle == 1 and ring == 1 and pinky == 1:
        return "5 / Open Hand", (255, 255, 0)

    # ── Fist / 'S' ── all fingers curled
    if thumb == 0 and index == 0 and middle == 0 and ring == 0 and pinky == 0:
        return "S / Fist", (100, 100, 255)

    return "Unknown", (180, 180, 180)

# ─── Draw UI ─────────────────────────────────────────────
def draw_ui(frame, sign, color, fingers, confidence_frames, total_frames=10):
    h, w, _ = frame.shape

    # Big sign display box
    cv2.rectangle(frame, (w-220, 10), (w-10, 120), (30, 30, 30), -1)
    cv2.rectangle(frame, (w-220, 10), (w-10, 120), color, 3)
    cv2.putText(frame, "Sign:", (w-210, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 1)
    cv2.putText(frame, sign, (w-210, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.6, color, 3)

    # Finger state indicators
    finger_names = ["T", "I", "M", "R", "P"]
    for i, (name, state) in enumerate(zip(finger_names, fingers)):
        x = 10 + i * 50
        c = (0, 255, 0) if state else (0, 0, 255)
        cv2.circle(frame, (x + 15, h - 40), 18, c, -1)
        cv2.putText(frame, name, (x + 7, h - 33), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,0,0), 2)

    # Stability bar (shows how consistently you're holding the sign)
    bar_fill = int((confidence_frames / total_frames) * 200)
    cv2.rectangle(frame, (10, h-80), (210, h-60), (50,50,50), -1)
    cv2.rectangle(frame, (10, h-80), (10 + bar_fill, h-60), (0, 255, 150), -1)
    cv2.putText(frame, "Hold steady", (10, h-85), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (180,180,180), 1)

    return frame

# ─── MAIN ────────────────────────────────────────────────
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
) as hands:

    cap = cv2.VideoCapture(1)

    last_sign = "Unknown"
    confidence_frames = 0
    STABILITY_THRESHOLD = 8  # frames sign must be stable before confirming

    print("Show ASL signs to the camera — press 'q' to quit")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                fingers = get_finger_states(landmarks)

                # Classify
                sign, color = classify_sign(landmarks)

                # Stability tracking — sign must hold for N frames
                if sign == last_sign:
                    confidence_frames = min(confidence_frames + 1, STABILITY_THRESHOLD)
                else:
                    confidence_frames = 0
                    last_sign = sign

                frame = draw_ui(frame, sign, color, fingers, confidence_frames, STABILITY_THRESHOLD)
        else:
            cv2.putText(frame, "Show your hand to the camera", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 200, 255), 2)
            confidence_frames = 0

        cv2.imshow("ASL Sign Detector", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Show ASL signs to the camera — press 'q' to quit
